In [21]:
from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast, save
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bloqade.emulate.ir.state_vector import StateVector, RydbergHamiltonian
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

output_notebook()

Loading BokehJS ...

In [22]:
Rb = np.sqrt(2*np.sqrt(5))*4
C6 = 862690*2*np.pi
Ω_max = 2.5*2*np.pi
Δ_end = C6 / (Rb**6)
Δ_start = -10*2*np.pi
Δ0 = Δ_end
T_max = 0.8

In [23]:
#geometry
rng = np.random.default_rng(18)
atom_pos=Square(4, lattice_spacing=4).apply_defect_density(0.2, rng=rng).remove_vacant_sites()
atom_pos.show()

In [25]:
x = [-1.107228162761049, 0.5642920574603194, 0.7980133135130201, 2.139708978471062, 7.9843742484094555]

#dynamics
durations_delta = [0.05, 0.15, 0.1, 0.1, 0.1, 0.1, 0.15, 0.05]
durations_delta = [5*i for i in durations_delta]
deltas = [Δ_start, Δ_start, Δ0*x[0], Δ0*x[1], Δ0*x[2], Δ0*x[3], Δ0*x[4], Δ_end, Δ_end]
durations_omega = [0.1, 0.1, 3.6, 0.1, 0.1]
omegas = [0.0, 0.0, Ω_max, Ω_max, 0.0, 0.0]

Delta = piecewise_linear(durations_delta,deltas)
Omega = piecewise_linear(durations_omega,omegas)

#create Hamiltonian
program = rydberg_h(atom_pos, detuning=Delta, amplitude=Omega, phase=None)
#output = program.bloqade.python().run(shots=100, interaction_picture=True)
output_Q = program.braket.aquila().run_async(shots=100)
#output.report().counts()

In [27]:
#output.report().counts()
output_Q.fetch()

                                             task ID     status  shots
0  arn:aws:braket:us-east-1:592242689881:quantum-...  Completed    100

In [28]:
output_Q.report().counts()

[OrderedDict([('1111111111110', 6),
              ('1111111111111', 6),
              ('1111110111111', 5),
              ('0111111111011', 4),
              ('1110111111111', 4),
              ('1111111111010', 4),
              ('1111111111011', 4),
              ('1110111111110', 3),
              ('1111110111011', 3),
              ('1111111011111', 3),
              ('1111111110011', 3),
              ('0111111110011', 2),
              ('0111111111101', 2),
              ('1011111011111', 2),
              ('1011111111011', 2),
              ('1011111111111', 2),
              ('1110110111111', 2),
              ('1110111110111', 2),
              ('1111010111110', 2),
              ('1111011111111', 2),
              ('1111111111101', 2),
              ('0010000011010', 1),
              ('0110010010010', 1),
              ('0111010111110', 1),
              ('0111101111011', 1),
              ('0111110011100', 1),
              ('0111110101111', 1),
              ('011111011110

In [25]:
save(output_Q, "outputQ_5x5")

In [29]:
output_Q.report().show()